#verify pyEMU Influence class

In [27]:
%matplotlib inline
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyemu

instaniate ```pyemu``` object and drop prior info.  Then reorder the jacobian and save as binary.  This is needed because the pest utilities require strict order between the control file and jacobian

In [40]:
pst = pyemu.Pst("freyberg.pst")
pst.pestpp_options = {}
inf = pyemu.Influence(jco="freyberg.jcb",pst=pst,verbose=False)
inf.drop_prior_information()



pick just a few parameters since infstat is super picky about this

In [41]:
par = inf.pst.parameter_data
adj_pars = par.loc[par.pargp=="hk","parnme"][:10].values.tolist()
fixed_pars = [pname for pname in inf.pst.adj_par_names if pname not in adj_pars]
jco_ord = inf.jco.get(inf.pst.obs_names,adj_pars)
ord_base = "freyberg_ord_infstat"
inf.pst.parameter_data.loc[fixed_pars,"partrans"] = "fixed"
#jco_ord.to_binary(ord_base + ".jco")  
#inf.pst.control_data.parsaverun = ' '
inf.pst.control_data.noptmax = 0
inf.pst.write(ord_base+".pst")

run pest one time since it is super picky about the format of the .res file

In [42]:
os.system("pest.exe "+ord_base+'.pst')

0

Write the .jco since pest just stomped on it

In [43]:
jco_ord.to_binary(ord_base + ".jco") 

In [8]:
os.system("infstat freyberg_ord")

0